# Bidirectional LSTM for Named Entity Recognition

© Data Trainers LLC. GPL v 3.0.

**Author:** Axel Sirota


In this notebook we will train a Bi-LSTM on NER dataset to create the appropriate tags

Take it easy and notice the structure of the dataset to ensure for each word if it has a tag or not

You can run this lab both locally or in Colab.

- To run in Colab just go to `https://colab.research.google.com`, sign-in and you upload this notebook. Colab has GPU access for free.
- To run locally just run `jupyter notebook` and access the notebook in this lab. You would need to first install the requirements in `requirements.txt`

Follow the instructions. Good luck!

In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [2]:
!pip install textblob 'keras-nlp' 'keras-preprocessing' 'gensim==4.2.0' np_utils swifter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.1/590.1 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 70.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 90.9 MB/s eta 0:00:00
  Created wheel for np_utils: filename=np_utils-0.6.0-py3-none-any.whl size=56438 sha256=301861d3bf1ee4079b550e37b403df57ee7e4cef179d707278de6efb5ebf9b1b
  Stored in directory: /root/.cache/pip/wheels/b6/c7/50/2307607f44366dd021209f660045f8d51cb976514d30be7cc7
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16507 sha256=9ed318752

In [3]:
import multiprocessing
import tensorflow as tf
import sys
import keras.backend as K
from keras.models import Sequential
from tensorflow.keras import Model, Input
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Embedding, Lambda, ELU, Conv1D, MaxPooling1D, Dropout, LSTM, TimeDistributed, SpatialDropout1D, Bidirectional
import np_utils
from tensorflow.keras.utils import to_categorical
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from textblob import TextBlob, Word
from keras_preprocessing.sequence import pad_sequences
from keras.initializers import Constant
from tensorflow.keras.layers.experimental import preprocessing
import keras_nlp
import os
import time
import sys
import numpy as np
import random
import os
import pandas as pd
import gensim
import warnings
import nltk
import pickle
from tensorflow.nn import leaky_relu

import re
import warnings
from sklearn.model_selection import train_test_split
from textblob import TextBlob
from collections import defaultdict


TRACE = False
embedding_dim = 100
rnn_units = 128
epochs=100
buffer_size = 256
max_len = 50
# Batch size
batch_size = 256
min_count_words = 3
BATCH = True

def set_seeds_and_trace():
  os.environ['PYTHONHASHSEED'] = '0'
  np.random.seed(42)
  tf.random.set_seed(42)
  random.seed(42)
  if TRACE:
    tf.debugging.set_log_device_placement(True)

def set_session_with_gpus_and_cores():
  cores = multiprocessing.cpu_count()
  gpus = len(tf.config.list_physical_devices('GPU'))
  config = tf.compat.v1.ConfigProto( device_count = {'GPU': gpus  , 'CPU': cores} , intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
  sess = tf.compat.v1.Session(config=config)
  tf.compat.v1.keras.backend.set_session(sess)

set_seeds_and_trace()
set_session_with_gpus_and_cores()
warnings.filterwarnings('ignore')
nltk.download('punkt')
textblob_tokenizer = lambda x: TextBlob(x).words

Using TensorFlow backend


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
%%writefile get_data.sh
if [ ! -f ner_dataset.csv ]; then
  wget -O ner_dataset.csv https://www.dropbox.com/s/mbfv0x988mdj89h/ner_dataset.csv?dl=0
fi


Writing get_data.sh


In [5]:
!bash get_data.sh

--2023-09-25 01:48:06--  https://www.dropbox.com/s/mbfv0x988mdj89h/ner_dataset.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/mbfv0x988mdj89h/ner_dataset.csv [following]
--2023-09-25 01:48:07--  https://www.dropbox.com/s/raw/mbfv0x988mdj89h/ner_dataset.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7d1f2c8eea5c1790b0d68f4c57.dl.dropboxusercontent.com/cd/0/inline/CEWLE2o04dKgVHdXTr9F6DK-G-BY7iDetf-rtonDKDQQ0s1xfzmpc8jzWewTsnypcpy0XTpNufH-HN-ikIm1b7YTd94cJY5NBbRHozFhRydrBC7WODSFCeirhportY8xgqVSPK9j8hlqe7fRerMKNfJ4/file# [following]
--2023-09-25 01:48:07--  https://uc7d1f2c8eea5c1790b0d68f4c57.dl.dropboxusercontent.com/cd/0/inline/CEWLE2o04dKgVHdXTr9F6DK-G-BY7iDetf-rtonDKDQQ0s1xfzmpc8jzWewTsnypcpy0XTpNufH-HN-ikIm1b7YT

In [ ]:
data= pd.read_csv("./ner_dataset.csv",encoding="latin1")
data = None # Forward Fill NaN
data.head()

In [ ]:
print("Unique Words in corpus:",data['Word'].nunique())
print("Unique Tag in corpus:",data['Tag'].nunique())

In [ ]:
words = None # Get list of words
words.append("ENDPAD")
num_words = len(words)
tags = None # Get list of tags
num_tags = len(tags)

In [ ]:
class SentenceGetter(object):
  def __init__(self,data):
    self.n_sent = 1 #counter
    self.data = data
    agg_func = lambda s:[(w,p,t) for w,p,t in zip(s['Word'].tolist(),s['POS'].tolist(),s['Tag'].tolist())]
    self.grouped = self.data.groupby("Sentence #").apply(agg_func)
    self.sentences = [s for s in self.grouped]



getter = SentenceGetter(data)
sentences = getter.sentences

In [ ]:
sentences[0]

In [ ]:
word2idx =  {w : i+1 for i,w in enumerate(words)}
tag2idx  =  {t : i for i,t in enumerate(tags)}

In [ ]:
X = None # Construct tensor of X with the ids of words and pad to the right

y = None # Construct tensor of y with the ids of tags and pad to the right
y = [to_categorical(i, num_classes=num_tags) for i in y]
x_train,x_test,y_train,y_test = train_test_split(X,y, test_size=0.1, random_state=1)


In [ ]:
model = Sequential()
model.add(Input(shape = (max_len,)))
model.add() # Add embedding layer
model.add()  # Add SpatialDropout
model.add()  # Add a Bi-LSTM with 100 units and a recurrent dropout for forgetfulness
model.add(TimeDistributed(Dense(num_tags,activation='softmax')))

In [ ]:
# Compile the model, use  kullback leibler divergence as metric (we will talk why)
model.summary()

In [ ]:
# Train the model with early stopping if the validation kullback leibler metrics goes up, and return the best weights!

In [ ]:
# Evaluate the model

In [ ]:
i = np.random.randint(0, x_test.shape[0])
p = model.predict(np.array([x_test[i]]))
p = np.argmax(p, axis=-1)

y_true = np.argmax(np.array(y_test), axis=-1)[i]
print("{:15}{:5}\t {}\n".format("Word","True","Pred"))
print("-"*30)
for w,true,pred in zip(x_test[i], y_true, p[0]):
  print("{:15}{:5}\t{}".format(words[w-1], tags[true],tags[pred]))
